# Basic code
## Introduction
This section introduces point cloud format and functions used frequently.

### TL;DR
- This tutorial uses the point cloud format of a two-dimensional array (shape: (number of points, channel size)).
- This tutorial frequently uses `tutlibs.io.Points` (reading and writing a file) and `tutlibs.visualization.JupyterVisualizer` (visualization for 3D representation).


## Format
This subsection introduces point cloud format. In this subsection, we use this package:

In [2]:
import numpy as np

Point cloud processing treats a point cloud as an array. As the following example, an array shows a point cloud with four points on three dimension space:

In [3]:
coords = np.array([
    [1, 0, 0],
    [0, 0.5, 1],
    [1, 1, 0.1],
    [-1, 0.5, -0.3]
], dtype=np.float32)

print('array shape:{}'.format(coords.shape))


array shape:(4, 3)


The first dimension of this array (`coords`) has points, and the second dimension has coordinate values of each point. Therefore, `4` of the array shape show number of points, and `3` show number of coordinate values on three dimension space.

A point cloud may contain additional information such as color information. If a color is assigned to each point in the above code, the color information is expressed as follows.

In [4]:
colors = np.array([
    [255, 10, 20],
    [0, 89, 111],
    [74, 116, 121],
    [190, 95, 3]
], dtype=np.int32)

print('array shape:{}'.format(colors.shape))

array shape:(4, 3)


The first dimension of this array (`colors`) has points, and the second dimension has color values of each point. Therefore, `4` of the array shape show number of points, and `3` show number of color values on three dimension space. Also, the index values of the first dimension correspond between the color and coordinate arrays. 

As you can see from the array of color and coordinate information, the point cloud array is represented in (number of points, channel size). The tutorial demonstrate various point cloud processing using this simple point cloud format. 

In this tutorial, the specifications of the point cloud array are as follows.
- This array shape is (number of points, channel size).
- Point cloud data is divided by type such as coordinates, colors.
- Arrays of the same point cloud correspond between the indices of the first dimension. (ex: i-th point color is `colors[i]`, coordinate is `coord[i]`.)

Note: 本チュートリアルでは(点の数, チャンネル数)を使用します。これ以外にも、画像ライクな形状である(width, height, channel size)があります。こちらは、[PCL](https://pcl.readthedocs.io/projects/tutorials/en/latest/basic_structures.html#basic-structures)で設けられています。また形状以外にも、チャンネルの取扱方法として一つの変数に各チャンネルをまとめる方法が採用されている場合もあります。例えば、[Open3D](http://www.open3d.org/docs/0.12.0/python_api/open3d.geometry.PointCloud.html#open3d.geometry.PointCloud.colors)は一つの点群の型に複数のチャンネルが格納されており、これらのチャンネル情報はクラス変数を介して各チャンネルにアクセスできるように設計されています。

## Function
This subsection introduces the following functions that the tutorial use frequently:
- point cloud file I/O (tutlibs.io.Points)
- visualization of point clouds (tutlibs.visualization.JupyterVisualizer)

In this subsection, we use this package:

In [1]:
import numpy as np
import os
from tutlibs.visualization import JupyterVisualizer as jv
from tutlibs.io import Points

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


### Point cloud file I/O (tutlibs.io.Points)
はじめに、点群ファイルの点群を`io.read`で読み取ります。コードは以下の通りです。

In [2]:
xyz, rgb, data = io.read('../data/bunny_pc.ply')
print('XYZ shape:{}'.format(xyz.shape))
print('RGB shape:{}'.format(rgb.shape))
print('data type:{}, key:{}'.format(type(data), data.keys()))

XYZ shape:(3000, 3)
RGB shape:(3000, 3)
data type:<class 'dict'>, key:dict_keys(['nx', 'ny', 'nz'])


`io.read`は`ply`ファイルからデータを読み取ります。注意点として、plyファイルはメッシュデータを取り扱うファイルですが、メッシュデータの頂点を点群とみなすことで、ここではplyファイルを点群ファイルの様に取り扱っています。
`io.read`の返り値は(`xyz`, `rgb`, `data`)の3つです。返り値の説明は以下のとおりです。
1. `xyz`: 点群の座標情報が入ります。この関数ではYXZ値を取得するため、受け取る値は(点の数, 3)となります。
2. `rgb`: 点群の色情報が入ります。この関数ではファイル内のRGB値を色情報として読み込むため、色情報がある場合は(点の数, 3)を返します。色情報がない場合はNoneを返します。
3. `data`: ファイル内のデータを辞書形式で保存します。一部sectionでのみ使用します。

次に、この点群を`io.write`でplyファイルに保存します。コードは以下の通りです。

In [3]:
os.makedirs('outputs/',exist_ok=True)
io.write('outputs/basic_code_pc.ply', xyz, rgb, color_range=[0, 255],
         additional_data=data)

`io.write`は`ply`ファイルへ点群を保存します。保存する際に必要となる値は以下のとおりです。
- `filename`: 保存ファイルパス
- `xyz`: 点群の座標情報, 入力配列の形状は(点の数, 3)となります。
- `rgb`: 点群の色情報, 入力配列の形状は(点の数, 3)となります。初期値はNoneであり、入力がNoneの場合は色情報を保存しません。
- `color_range`: 色情報の値の範囲を入力します。初期値は[0, 1]であり、これは入力した色情報(RGB値)が0~1の範囲で示されている場合の設定となります。もしRGB値の範囲が0~255の範囲である場合は、[0, 255]となります。
- `additional_data`: 追加で保存するデータを辞書形式で入力します。初期値はNoneであり、Noneの場合は追加の情報は入力されません。基本は使いません。


### 視覚化 (tutlibs.visualization.JupyterVisualizer as jv)
最後に、点群を視覚化します。コードは以下のとおりです。

In [4]:
obj_point = jv.point(xyz, rgb, color_range=[0, 255])
jv.display([obj_point])

Output()

`jv.point`を用いることで、視覚化するための変数を用意する。`jv.point`で主に使う引数は以下の通り。
- `xyz`: 視覚化する点群の座標情報, 入力配列の形状は(点の数, 3)。
- `rgb`: 視覚化する点群の色情報, 入力配列の形状は(点の数, 3)。初期値はNoneであり、入力がNoneの場合は色情報を反映せず、点群を青で表示する。
- `color_range`: 色情報の値の範囲を入力する。初期値は[0, 1]であり、これは入力した色情報(RGB値)が0~1の範囲で示されている場合の設定となる。もしRGB値の範囲が0~255の範囲である場合は、[0, 255]となる。

`jv.display`は、`jv.point`等で用意された視覚化用変数のリストを、全て表示する。

## 開発に関するTopic
### TL;DL;
- 点群のコードを書く場合は、型とデータ形状のヒントを載せることを推奨します。
- 本チュートリアルでは、コメントと[typing](https://docs.python.org/ja/3/library/typing.html)によるヒントを使います。

### 説明
上記では点群のチャンネルとして座標値と色情報を取り扱いましたが、点群ではこれら以外のチャンネルを扱うこともあります。例えば、LiDARと呼ばれるセンサーからは反射強度を得ることがあります。また、他の3D表現データから点群をサンプリングした場合、法線情報も取得できる場合があります。  
また、チャンネルの変化以外にも、処理の過程でデータの形状がいくらか変化することが多いです。例えば、kNNによって点ごとの近傍を得る場合、データの形状は(点の数, kNNの近傍数, チャンネル数)といった形に変化します。

この様に、点群データの変数はいろいろと変化します。この変化は共同開発や再利用時にコードの理解を妨げる要因になりやすいです。従って、実際に点群データを使用するコードを書く場合は、関数の引数や返り値などに[通常のヒント](https://docs.python.org/3/library/typing.html)と形状に関するヒントを載せておくことを強く推奨します。ここの「形状に関するヒント」とは、(点の数, チャンネル数)のことを指します。

これらのヒントはコメントや[typing](https://docs.python.org/ja/3/library/typing.html)以外にも、[numpy typing](https://numpy.org/devdocs/reference/typing.html)、[torchtyping](https://github.com/patrick-kidger/torchtyping)で示すことができます。ただし、2021/09/05現在、本チュートリアルではコメントと[typing](https://docs.python.org/ja/3/library/typing.html)によるヒントの実装を行います。理由としては、
- 実装者本人の慣れ
- 実装時期(typing以外が新しすぎる)